In [2]:
from braindecode.datasets.moabb import MOABBDataset

subject_id = [1,2,3,4,5,6,7,8,9]
#subject_id = [1]
#2a= BNCI2014001 2b= BNCI2014004
dataset = MOABBDataset(dataset_name="BNCI2014001",subject_ids=subject_id)
print(dataset)


48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
48 events found
Event IDs: [1 2 3 4]
4

In [ ]:
dataset.datasets[4]
dataset.save("dataset\\2amoabb\\train")

In [5]:
import moabb
from braindecode.datasets.moabb import MOABBDataset
dataset = moabb.datasets.PhysionetMI(imagined = True, executed = True)
subject_id = list(range(1,110))
dataset.download(subject_list=subject_id,path="dataset\\physio")

In [ ]:
dataset.data_path(2)

In [ ]:
import mne
print(mne.get_config())
print(mne.get_config_path())

In [7]:
print(list(range(1,110)))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109]


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf,read_raw_edf,read_raw_gdf,read_raw_fif
from mne.datasets import eegbci
from mne.decoding import CSP
path = "dataset\\physio\\MNE-eegbci-data\\files\\eegmmidb\\1.0.0"
folders = os.listdir(path)
subject_count = 0

data_path = []
for fol in folders:
    file = os.path.join(path,fol)
    subfols = os.listdir(file)
    print(fol)
    for f in subfols:
        file_data = os.path.join(file,f)
        #print(file_data)
        data_path.append(file_data)
    subject_count += 1
    if subject_count == 10:
        break

S001
S002
S003
S004
S005
S006
S007
S008
S009
S010


In [18]:
def get_data():
    path = "dataset\\physio\\MNE-eegbci-data\\files\\eegmmidb\\1.0.0"
    folders = os.listdir(path)
    subject_count = 0

    data_path = []
    for fol in folders:
        file = os.path.join(path,fol)
        subfols = os.listdir(file)
        for f in subfols:
            file_data = os.path.join(file,f)
            print(file_data)
            data_path.append(file_data)
        subject_count += 1
        if subject_count == 10:
            break
        
    
    #tmin, tmax = -0.2, 0.4
    tmin, tmax = 1, 2
    #event_id = dict(left_hand=2, right_hand=3, feet=5, hands=4, rest=1)

    #first is all data path second is one file
    #All subject
    raw = concatenate_raws([read_raw_edf(f, preload=True) for f in data_path])
    #one subject
    #raw = concatenate_raws([read_raw_fif(test_path, preload=True)])
    
    
    raw_data = raw.copy()
    print(raw_data.info)
    # strip channel names of "." characters
    raw_data.rename_channels(lambda x: x.strip('.'))

    # Apply band-pass filter
    #skip_by_annotation='edge'
    raw_data.filter(l_freq=0.5, h_freq=60,skip_by_annotation='edge')
    raw_data.notch_filter(50)

    events, event_id = events_from_annotations(raw_data)
    print(events)
    print(event_id)
    
    picks = pick_types(raw_data.info, meg=False, eeg=True, stim=False, eog=False,
                       exclude='bads')
    print(picks)
    #delete eog band
    #picks = np.delete(picks,[3,4,5])
    
    # Read epochs (train will be done only between 1 and 2s)
    # reject_criteria = dict( eeg=100e-6)       # 100 µV
    epochs = Epochs(raw_data, events, event_id, tmin, tmax, proj=True, picks=picks,
                    baseline=None,preload=True)
    #epochs.crop(tmin=1., tmax=None)
    labels = epochs.events[:, -1]-1
    return epochs.get_data(), labels,epochs,raw_data


epochs_data, labels,epochs1,raw= get_data()

dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R01.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R02.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R04.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R06.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R08.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R10.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R12.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R14.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S002\S002R01.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S002\S002R02.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S002\S002R04.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S002\S002R06.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S002\S002R08.edf
dataset\physio\MNE-eegbci-data\files\eegmmidb\1.0.0\S002\S002R10.edf
dataset\physio\MNE-eegbci-data\fil

In [1]:
# 280 events(epoch)
# 22 channel
# 256 Time(samples)
#(event,channel,time)
print(epochs_data.shape)
print(labels.shape)
print(epochs_data.shape[1])
print(labels)
print(raw.info['ch_names'])
print(raw.info['nchan'])

NameError: name 'epochs_data' is not defined